## Simpy
Ejercicios para aprender a usar ambientes en python con la BIBLIOTECA simpy, con sistemas discretos. Importamos simpy

In [ ]:
import simpy

El objeto provee una colección de métodos que nos permite controlar el ambiente, donde todo ocurre incluyendo el tiempo (de la ejecución), lo podríamos ver como escenario. 


In [ ]:
env = simpy.Environment() #Creamos el ambiente con simpy
print('time=', env.now) #es el tiempo 0
env.run() #Así lo corremos aunque no hemos puesto nada en la simulacion

Los procesos con componentes activos que se implemetan con yield y se adjuntan con env.process.

Un proceso se comunica con el entorno con yield, es decir envia un evento al ambiente.

Los procesos se describen mediante los generadores de python

In [ ]:
def print_msg():
    yield env.timeout(500)
    print('time =', env.now,'; mensaje de prueba jsjs')

#Creamos el ambiente 
env = simpy.Environment()

#Usamos el generador para crear el proceso, que genera la funcion
env.process(print_msg())

#corremos la simulación
env.run()

También puede recibir argumentos para saber en qué momento correrlo, como le indicamos el momento entonces los imprimirá en orden

In [ ]:
def print_msg_param(demora, msg):
    yield env.timeout(demora)
    print(f'time {demora}: {msg}')

#Creamos el ambiente con simpy.Environment, y lo procesamos
env.process(print_msg_param(3, 'Este es el tercer mensaje'))
env.process(print_msg_param(1, 'Este es el primer mensaje'))
env.process(print_msg_param(2, 'Este es el segundo mensaje'))
env.process(print_msg_param(5, 'Este es el quinto mensaje'))
env.process(print_msg_param(4, 'Este es el cuarto mensaje'))
#Se imprimiran en orden ya que le indicamos el tiempo
env.run()

Cuando un proceso produce un evento el proceso se suspende y simpy reanuda el procesos cuando ocurre un evento, es algo analogo con las rondas

## Ejemplos con carros
Definimos la función de carro, para simular el comportamiento de uno. El proceso requiere de una refrencia al entorno para crear nuevos eventos.
Aunque tenemos un ciclo while que nunca acaba recordemos que es un generador por lo cual la simulación se reanuda en el yield donde lo dejamos.

In [ ]:
def carro(env):
    ''' Proceso de carro
    env: Environment
    El ambiente en el que estará el proceso
    '''
    while True:
        print('El carro se para en el tiempo', env.now)
        duracion_p = 2
        yield env.timeout(duracion_p) #esperamos hasta que el proceso esté en el tiempo 5, pasen 5 segundos
    
        print('Empezamos a conducir en el tiempo', env.now)
        duracion_c = 7
        yield env.timeout(duracion_c) #esperamos otros 7 segundos (en realidad no son segundos)

env = simpy.Environment() #creación del ambiente
env.process(carro(env)) #Le agregamos el proceso a correr
env.run(until=25)

Al producir el evento, le indica a la simulación que quiere esperar a que ocurra el evento.

Otro ejemplo de carros un poco más elaborado con recursos compartidos

In [ ]:
def carro(env, nombre, ec, tiempo_conduccion, duracion_carga):
    '''
    env: Ambiente
    nombre: Nombre del proceso
    ec: Estación de carga
    tiempo_conduccion: Tiempo de condicción que queremos sar
    duracion_carga: Duración de la carga
    Ilustra el uso de los recursos compartidos
    '''
    yield env.timeout(tiempo_conduccion) #El tiempo en que llega
    print(f'{nombre} llegando en el tiempo {env.now}')

    with ec.request() as req:
        yield req

        print(f'{nombre} cargando en el tiempo {env.now}')
        yield env.timeout(duracion_carga) #cuanto tiempo queremos esperar
        print('%s dejando la EC en el tiempo %s' % (nombre, env.now))


env = simpy.Environment() #creamos ambiente
ec = simpy.Resource(env, capacity=2) #Definimos el recurso compartido y la capacidad

env.process(carro(env, 'Carro 1', ec, 1, 5))#añadimos el proceso que queremos correr
env.process(carro(env, 'Carro 2', ec, 1, 5))#llegan al mismo tiempo
env.process(carro(env, 'Carro 3', ec, 5, 5))#llega en el tiempo 5
env.process(carro(env, 'Carro 4', ec, 3, 10))#llega en el tiempo 5


env.run()#no hay procesos que corran pa siempre, no es necesario el until

La instancia de Process que devuelve Environment.process() se puede utilizar para interacciones de procesos. Los dos ejemplos más comunes de esto son esperar a que finalice otro proceso

In [ ]:
class Carro:
    '''Ilustra la espera de otros procesos'''
    def __init__(self, env) -> None:
        self.env = env
        self.action = env.process(self.run())

    def run(self):
        '''El proceso del carro que el ambiente va a ejecutar'''
        while True:
            print('Empieza a cargarse en el tiempo %d' % self.env.now)
            duracion_carga=5
            yield self.env.process(self.carga(duracion_carga)) #esperamos el proceso de carga
            #el proceso espera a que finalice

            print('Empieza a manejar en el tiempo %d' % self.env.now)
            duracion_viaje=12
            yield self.env.timeout(duracion_viaje)

    def carga(self, duracion):
        yield self.env.timeout(duracion)

env=simpy.Environment()
carro = Carro(env)
env.run(until=20)